<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Proyecto integrador (parte 3): Análisis exploratorio de datos </h3>

<p> Alumno </p>
<p> Fernando Luna Ponce </p>

<p> Profesor </p>
<p> Dr. Julio Waissman Vilanova </p>

</center>

Importamos las librerías necesarias.

In [201]:
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.request
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from pandas_profiling import ProfileReport
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import MultiColumnLabelEncoder

In [202]:
subdir = "./data/tidy_data/"

df_tidy = pd.read_parquet(subdir + 'Tidy_data.parquet')
df_tidy

,fechaRegistro,Nombre_Municipio,tloc_resid,ocupacion,descr_sexo,Edad,asist_medi,sitio_ocur,lengua
0,2017-02-24,San Luis Río Colorado,13,99,Hombre,72,1,4,9
1,2017-02-20,San Luis Río Colorado,13,8,Hombre,28,1,3,9
2,2017-01-20,San Luis Río Colorado,13,9,Hombre,56,1,3,9
3,2017-01-09,San Luis Río Colorado,13,11,Mujer,78,1,9,9
4,2017-02-11,San Luis Río Colorado,13,11,Mujer,71,1,3,1
...,...,...,...,...,...,...,...,...,...
9034,2021-12-15,Cajeme,14,11,Hombre,78,1,11,2
9035,2021-12-28,San Luis Río Colorado,13,5,Hombre,70,1,11,2
9036,2021-12-23,Navojoa,2,11,Mujer,61,2,3,2
9037,2021-12-15,San Luis Río Colorado,1,11,Mujer,61,1,11,2


In [203]:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9034 entries, 0 to 9038
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fechaRegistro     9034 non-null   datetime64[ns]
 1   Nombre_Municipio  9034 non-null   object        
 2   tloc_resid        9034 non-null   int64         
 3   ocupacion         9034 non-null   int64         
 4   descr_sexo        9034 non-null   object        
 5   Edad              9034 non-null   int64         
 6   asist_medi        9034 non-null   int64         
 7   sitio_ocur        9034 non-null   int64         
 8   lengua            9034 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 705.8+ KB


Métodos de agregación

In [204]:
df_tidy.describe()

,tloc_resid,ocupacion,Edad,asist_medi,sitio_ocur,lengua
count,9034.000000,9034.000000,9034.000000,9034.000000,9034.000000,9034.000000
mean,10.720390,16.224596,68.471552,1.477751,8.147111,3.102723
std,7.859018,23.374415,13.250539,1.712867,9.449239,2.627311
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,6.000000,11.000000,60.000000,1.000000,3.000000,2.000000
50%,13.000000,11.000000,69.000000,1.000000,11.000000,2.000000
75%,14.000000,11.000000,78.000000,1.000000,11.000000,2.000000
max,99.000000,99.000000,102.000000,9.000000,99.000000,9.000000


EDA automático

In [205]:
profile = ProfileReport(df_tidy, title="EDA_Defunciones_Diabetes")
profile.to_file("EDA_Defunciones_Diabetes.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [206]:
df_tidy['Nombre_Municipio'].value_counts()[:72].plot(kind='bar', figsize=(16,6), xlabel='Municipio', ylabel='Total de muertes')

<AxesSubplot:xlabel='Municipio', ylabel='Total de muertes'>

Series de tiempo

In [207]:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9034 entries, 0 to 9038
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fechaRegistro     9034 non-null   datetime64[ns]
 1   Nombre_Municipio  9034 non-null   object        
 2   tloc_resid        9034 non-null   int64         
 3   ocupacion         9034 non-null   int64         
 4   descr_sexo        9034 non-null   object        
 5   Edad              9034 non-null   int64         
 6   asist_medi        9034 non-null   int64         
 7   sitio_ocur        9034 non-null   int64         
 8   lengua            9034 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 705.8+ KB


In [208]:
date = np.array(df_tidy.fechaRegistro, dtype=np.datetime64)
date


array(['2017-02-24T00:00:00.000000000', '2017-02-20T00:00:00.000000000',
       '2017-01-20T00:00:00.000000000', ...,
       '2021-12-23T00:00:00.000000000', '2021-12-15T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000'], dtype='datetime64[ns]')

Método de reducción de características, PCA

In [209]:
Mcle = MultiColumnLabelEncoder.MultiColumnLabelEncoder()
df_pca = Mcle.fit_transform(df_tidy)

In [210]:
scaler = StandardScaler()
X_std = scaler.fit_transform(df_pca.iloc[:, 1:9])

In [211]:
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(X_std)
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8'])
principalDf

,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8
0,3.200777,0.249708,-0.750616,-0.925505,-0.749037,1.475979,2.198005,-0.704581
1,1.662188,1.050892,-2.314869,-0.278351,-0.886674,-2.474005,0.446933,0.574004
2,1.288212,0.860343,-0.887790,-0.477758,-1.692519,-1.262903,0.559467,0.280223
3,0.687334,0.488378,1.258474,-1.563066,-0.971727,-0.911590,1.026910,0.142856
4,-0.879095,0.721063,0.121792,-0.958442,0.577108,0.139820,-0.305944,-0.570378
...,...,...,...,...,...,...,...,...
9029,-0.324902,0.712728,0.238277,0.815788,-0.612671,0.672291,0.027194,-0.122864
9030,-0.302845,1.100118,-0.091279,0.811804,-0.523077,0.055882,-0.096145,-0.453953
9031,-0.417912,-0.785975,-0.289332,-0.679570,0.639388,-0.725986,-0.563872,-0.776297
9032,-0.576301,-0.192664,0.027982,-0.295658,0.780888,-1.100098,0.192657,-1.319295


Gráfica de la primera y segunda componente

In [212]:
fig = px.scatter(principalDf,
                 x='PCA_1', 
                 y='PCA_2', 
                 color_discrete_sequence=['blueviolet'], 
                 opacity=0.4
                )
fig.update_xaxes(ticks='outside', 
                 tickwidth=1, 
                 tickcolor='gray', 
                 ticklen=5)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    scaleanchor = 'x',
    constrain='domain'
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.update_traces(mode = 'markers', 
                  hovertemplate = None
)
fig.update_layout(hovermode='x unified')
fig.update_traces(
    hovertemplate =' ',
    showlegend = False,
)
fig.update_layout(
    hoverlabel = dict(
        bgcolor = 'LightSteelBlue',
        font_size = 40,
    )
)
fig.show()

Gráfica de las componentes 3 y 4

In [213]:
fig = px.scatter(principalDf,
                 x='PCA_3', 
                 y='PCA_4', 
                 color_discrete_sequence=['blueviolet'], 
                 opacity=0.4
                )
fig.update_xaxes(ticks='outside', 
                 tickwidth=1, 
                 tickcolor='gray', 
                 ticklen=5)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    scaleanchor = 'x',
    constrain='domain'
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.update_traces(mode = 'markers', 
                  hovertemplate = None
)
fig.update_layout(hovermode='x unified')
fig.update_traces(
    hovertemplate =' ',
    showlegend = False,
)
fig.update_layout(
    hoverlabel = dict(
        bgcolor = 'LightSteelBlue',
        font_size = 40,
    )
)
fig.show()

In [214]:
pd.DataFrame(pca.explained_variance_ratio_ , columns=['Varianza_Explicada'])

,Varianza_Explicada
0,0.167809
1,0.161735
2,0.142734
3,0.127207
4,0.111434
5,0.102120
6,0.098560
7,0.088402
